In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Bidirectional, Lambda
from sklearn.model_selection import train_test_split
import pickle
import numpy as np
from music21 import *
from copy import deepcopy
import os
import datetime

In [ ]:
# load the datasets and then split in to train/test data sets
# note that we don't need label sets, only input sets for bidirectional LSTM (labels == inputs)
# unidirectional label set is the same as the inputs but without the first time step
with open("pickles/short_sequences_duration.pickle", 'rb') as short_duration,\
     open("pickles/short_sequences_pitch.pickle", 'rb') as short_pitch:
    short_seqs_duration_train, short_seqs_duration_test = train_test_split(pickle.load(short_duration), train_size=0.95)
    short_seqs_pitch_train, short_seqs_pitch_test = train_test_split(pickle.load(short_pitch), train_size=0.95)

with open("pickles/medium_sequences_duration.pickle", 'rb') as medium_duration,\
     open("pickles/medium_sequences_pitch.pickle", 'rb') as medium_pitch:
    medium_seqs_duration_train, medium_seqs_duration_test = train_test_split(pickle.load(medium_duration), train_size=0.95)
    medium_seqs_pitch_train, medium_seqs_pitch_test = train_test_split(pickle.load(medium_pitch), train_size=0.95)
    
with open("pickles/long_sequences_duration.pickle", 'rb') as long_duration,\
     open("pickles/long_sequences_pitch.pickle", 'rb') as long_pitch:
    long_seqs_duration_train, long_seqs_duration_test = train_test_split(pickle.load(long_duration), train_size=0.95)
    long_seqs_pitch_train, long_seqs_pitch_test = train_test_split(pickle.load(long_pitch), train_size=0.95)

In [ ]:
# retrieve the pitches and durations that were used to build the data set
# these will be used to convert the output one-hot vectors back to actual pitch/duration values
with open('pickles/durations.pickle', 'rb') as d, open('pickles/pitches.pickle', 'rb') as p:
    durations = pickle.load(d)
    pitches = pickle.load(p)

num_durations = len(durations)
num_pitches = len(pitches)
short_seq_len = short_seqs_duration_train.shape[1]
medium_seq_len = medium_seqs_duration_train.shape[1]
long_seq_len = long_seqs_duration_train.shape[1]

In [ ]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

# defines the function that will be used to transform the input before it is recurrently fed in
# param. vector: one hot output of the previous time step after being passed through softmax activation func.
@tf.function
def sampling_activation(vector):
    # vector should be 1D array
    num_categories = vector.shape[0]
    # converts the vector to a distribution, samples from it, returns the index of the value
    # for some reason, thos does not work properly if I don't take the log of the vector
    index = tf.random.categorical(softmax(vector.reshape((1, num_categories))), 1).numpy()[0][0]
    ret_arr = np.zeros(num_categories)
    ret_arr[index] = 1.0
    return ret_arr

In [ ]:
logits = np.array([1,2,3,4])
logits_temp_p0 = logits / 0.01
logits_temp_p2 = logits / 0.2
logits_temp_p4 = logits / 0.4
logits_temp_p6 = logits / 0.6
logits_temp_p8 = logits / 0.8
logits_temp_p10 = logits / 1.0
logits_temp_p15 = logits / 1.5
logits_temp_p150 = logits / 150

print(f"RAW: {softmax(logits)}")
print(f"TEMP 0.0: {softmax(logits_temp_p0)}")
print(f"TEMP 0.2: {softmax(logits_temp_p2)}")
print(f"TEMP 0.4: {softmax(logits_temp_p4)}")
print(f"TEMP 0.6: {softmax(logits_temp_p6)}")
print(f"TEMP 0.8: {softmax(logits_temp_p8)}")
print(f"TEMP 1.0: {softmax(logits_temp_p10)}")
print(f"TEMP 1.5: {softmax(logits_temp_p15)}")
print(f"TEMP 150: {softmax(logits_temp_p150)}")

In [ ]:
# takes a 1D vector (a single sam) and converts it to 3D for input to the network
def timestep_to3d(x):
    return np.reshape(x, (1, 1, x.shape[0]))

# create one-hot vector representation for a time step given the index position of the encoded value
def vectorize(index, vec_size):
    index = int(index)
    vec = np.zeros(vec_size, np.float32)
    vec[index] = 1.0
    return vec

# returns the index of the one-hot encoded value
def unvectorize(x):
    return np.argmax(x)

In [ ]:
# single layer Unidirectional or Bidirectional LSTM; will easily allow us to test various configurations
def getModel(num_features, bidirectional=True, temperature=1.0):
    model = Sequential()
    # only dif. betwn. bi. LSTM and uni. LSTM is the presence/absence of Bidirectional wrapper
    # hidden layer 1; 20  units; input (# timesteps, # features); return a sequence of each time step's outputs
    # input_shape first value None makes it variable (we don't have fixed length sequences)
    # output of LSTM cell uses tanh activation, recurrent connections use sigmoid
    if bidirectional:
        model.add(Bidirectional(LSTM(20, input_shape=(None, num_features), return_sequences=True)))
    else:
        model.add(LSTM(20, input_shape=(None, num_features), return_sequences=True))
        
    # so that we can divibe by temperature before feeding through softmax
    model.add(Lambda(lambda x: x / temperature))
        
    # TimeDistributed is a wrapper allowing one output per time step; 
    # ...requires hidden layer to have return_sequences == True
    model.add(TimeDistributed(Dense(num_features, activation='softmax')))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', 'categorical_crossentropy'])
    return model

In [ ]:
# train LSTM
def trainModel(model, X, epochs=30, bidirectional=True):
    print(X.shape)
    Y = deepcopy(X)
    if not bidirectional:
        Y = Y[1:] # labels include all time steps but the first one in unidir. LSTM
    model.fit(X, Y, epochs=epochs, verbose=1, validation_split=0.2)

In [ ]:
# return a single index from a sample of a softmax distribution vector
def sample_distribution(dist_vec, num_categories):
    return tf.random.categorical(dist_vec.reshape(1, num_categories), 1).numpy().flatten()[0]

# use the trained model to compose new music by feeding in a single input and desired length
def compose(pitch_model, duration_model, pitch_prompt, duration_prompt, length):
    '''pitch_model: the trained model for pitch predictions
       duration_model: trained model for duration predictions
       pitch_prompt: the first pitch of the piece (index of the one-hot encoded pitch vector)
       duration_prompt: the first duration of the piece (index of the one-hot encoded duration vector)
       length: how many time steps to generate for
       
       returns a music21.stream.Stream object representing the composition
    '''    
    # the lists that hold the indices of the values to index in to pitches/durations lists
    generated_pitches, generated_durations = [pitch_prompt], [duration_prompt]
    
    current_pitch, current_duration = pitch_prompt, duration_prompt
    for t in range(length):
        # model only accepts 3D inputs
        pitch_vec = timestep_to3d(vectorize(current_pitch, num_pitches))
        duration_vec = timestep_to3d(vectorize(current_duration, num_durations))
        
        # predict the output distributions
        pitch_pred = pitch_model.predict(pitch_vec)
        duration_pred = duration_model.predict(duration_vec)
        # sample the distributions (returns the index of the one-hot vectors)
        next_pitch = sample_distribution(pitch_pred, num_pitches)
        next_duration = sample_distribution(duration_pred, num_durations)
        generated_pitches.append(next_pitch)
        generated_durations.append(next_duration)
        
        # get ready for next iteration
        current_pitch, current_duration = next_pitch, next_duration
        
    
    composed_stream = stream.Stream()
    for pair in list(zip(generated_pitches, generated_durations)):
        p = pitch.Pitch(midi=pair[0])
        d = duration.Duration(float(pair[1]))
        n = note.Note()
        n.pitch = p
        n.duration = d
        composed_stream.append(n)
        
    return composed_stream

In [ ]:
duration_bidirectional = getModel(num_durations, bidirectional=True)
trainModel(duration_bidirectional, short_seqs_duration_train, bidirectional=True)
trainModel(duration_bidirectional, medium_seqs_duration_train, bidirectional=True)
trainModel(duration_bidirectional, long_seqs_duration_train, bidirectional=True)

In [ ]:
pitch_bidirectional = getModel(num_pitches, bidirectional=True)
trainModel(pitch_bidirectional, short_seqs_pitch_train, bidirectional=True)
trainModel(pitch_bidirectional, medium_seqs_pitch_train, bidirectional=True)
trainModel(pitch_bidirectional, long_seqs_pitch_train, bidirectional=True)

In [ ]:
piece1 = compose(pitch_bidirectional, duration_bidirectional, 3, 4, 100)
piece2 = compose(pitch_bidirectional, duration_bidirectional, 3, 4, 100)

In [ ]:
piece1.show('midi')

In [ ]:
piece1.show()

In [ ]:
piece2.show('midi')

In [ ]:
piece2.show()

In [ ]:
duration_pred = duration_bidirectional.predict(to_3D(long_seqs_duration_test[0]))
composed_durations = duration_one_hot_to_quarter_length(duration_pred.reshape(long_seq_len, num_durations))

pitch_pred = pitch_bidirectional.predict(to_3D(long_seqs_pitch_test[0]))
composed_pitches = pitch_one_hot_to_MIDI(pitch_pred.reshape(long_seq_len, num_pitches))

In [ ]:
composed_pairs = list(zip(composed_pitches, composed_durations))

composed_stream = stream.Stream()
for pair in composed_pairs:
    p = pitch.Pitch(midi=pair[0])
    d = duration.Duration(pair[1])
    n = note.Note()
    n.pitch = p
    n.duration = d
    composed_stream.append(n)

In [ ]:
composed_stream.show('midi')

In [ ]:
composed_stream.show()